# Importing Required Library

In [113]:
## import numpy and pandas
import pandas as pd
import numpy as np

## import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## do an inline so that plt.show() is not required everytime
%matplotlib inline

## import for pre-processing
## scale (this is same as standard scalar)
from sklearn.preprocessing import scale
## label encoding
from sklearn.preprocessing import LabelEncoder

## import for model selection:
## splitting data into training and test data (validation set)
from sklearn.model_selection import train_test_split
## import for creating K-folds
from sklearn.model_selection import KFold
## import for CV using folds created
from sklearn.model_selection import cross_val_score
## import for grid search cv for Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

## for deleting unused variables
import gc

## filter warnings
import os
import warnings
warnings.filterwarnings('ignore')

# Until fuction: line seperator
def print_dashes_and_ln():
    print('-'*100, '\n')
    
# Formatter to display all float format in 2 decimal format
pd.options.display.float_format = '{:.2f}'.format

# Loading data from csv file

In [114]:
# Loading telecom churn data from telecom_churn_data.csv file
telecom_orig = pd.read_csv('telecom_churn_data.csv')
telecom = telecom_orig.copy()
print('Dataframe Shape: ', telecom.shape); print_dashes_and_ln();
telecom.head()

Dataframe Shape:  (99999, 226)
---------------------------------------------------------------------------------------------------- 



,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.00,0.00,0.00,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.38,...,0,1.00,1.00,1.00,nan,968,30.40,0.00,101.20,3.58
1,7001865778,109,0.00,0.00,0.00,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.05,...,0,nan,1.00,1.00,nan,1006,0.00,0.00,0.00,0.00
2,7001625959,109,0.00,0.00,0.00,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.69,...,0,nan,nan,nan,1.00,1103,0.00,0.00,4.17,0.00
3,7001204172,109,0.00,0.00,0.00,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.34,...,0,nan,nan,nan,nan,2491,0.00,0.00,0.00,0.00
4,7000142493,109,0.00,0.00,0.00,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.64,...,0,0.00,nan,nan,nan,1526,0.00,0.00,0.00,0.00


In [115]:
# Understanding data type, columns, amount, etc
print('Data related Information: ', telecom.info()); print_dashes_and_ln();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB
Data related Information:  None
---------------------------------------------------------------------------------------------------- 



After doing analysis on data and understanding business. We may come to understancd that those columns are more important. Whose column name contain rech_amt, rech_data, night_pck, fb_user as substring . So, firstly filling those columns with 0 as new category will help in further manupulation, cleaning and to determine the high value customers.

In [123]:
# let us first extract list of columns containing recharge amount
imp_columns =  telecom.columns[telecom.columns.str.contains('rech_amt|rech_data|user')]

print(imp_columns);print_dashes_and_ln();

Index(['total_rech_amt_6', 'total_rech_amt_7', 'total_rech_amt_8',
       'total_rech_amt_9', 'max_rech_amt_6', 'max_rech_amt_7',
       'max_rech_amt_8', 'max_rech_amt_9', 'date_of_last_rech_data_6',
       'date_of_last_rech_data_7', 'date_of_last_rech_data_8',
       'date_of_last_rech_data_9', 'total_rech_data_6', 'total_rech_data_7',
       'total_rech_data_8', 'total_rech_data_9', 'max_rech_data_6',
       'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9',
       'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8',
       'av_rech_amt_data_9', 'night_pck_user_6', 'night_pck_user_7',
       'night_pck_user_8', 'night_pck_user_9', 'fb_user_6', 'fb_user_7',
       'fb_user_8', 'fb_user_9'],
      dtype='object')
---------------------------------------------------------------------------------------------------- 



# Data Cleaning and Filling missing value

### Treating all Categorical Variables

In [95]:
#1. Data Cleaning. ## Replacing Columns values with -1 as new category when > 70% Null Values
#print('Total number of missing values in the DataFrame: ', telecom.isnull().values.sum());print_dashes_and_ln();
#print('Checking percentage the column-wise distribution of null values: ', round(100*(telecom.isnull().sum()/len(telecom.index)), 2));print_dashes_and_ln();
#categorical_vars = list(categorical_vars[categorical_vars.values>=0.7].index)
#print('The columns that need to replace with new category as -1: ', colnames);print_dashes_and_ln();
# Missing values can be imputed with '-1' 
# where -1 can be treated as a new category.
#for var in categorical_vars:
#    telecom[var].fillna(value=-1, inplace=True)
    
    
#1. Data Cleaning. Removing Columns with > 70% Null Values
colnames = (telecom.isnull().sum()/len(telecom))
colnames = list(colnames[colnames.values>=0.7].index)
telecom.drop(labels = colnames,axis =1,inplace=True)
print("Number of Columns dropped\t: ",len(colnames));print_dashes_and_ln();

Number of Columns dropped	:  40
---------------------------------------------------------------------------------------------------- 



In [96]:
#2. Data Cleaning. ## Removing Columns with single Unique Value
unique = telecom.nunique()
unique = unique[unique.values == 1]
telecom.drop(labels = list(unique.index), axis =1, inplace=True)
print("So now we are left with",telecom.shape ,"rows & columns."); print_dashes_and_ln();

So now we are left with (99999, 170) rows & columns.
---------------------------------------------------------------------------------------------------- 



In [97]:
#3. Columns by Data Type
non_numeric_columns = telecom.select_dtypes(['object']).columns
numeric_columns = telecom.select_dtypes(['number']).columns
print("Numeric \t: ",numeric_columns); print_dashes_and_ln();
print("Non-numeric Columns \t: ",non_numeric_columns); print_dashes_and_ln();



Numeric 	:  Index(['mobile_number', 'arpu_6', 'arpu_7', 'arpu_8', 'arpu_9', 'onnet_mou_6',
       'onnet_mou_7', 'onnet_mou_8', 'onnet_mou_9', 'offnet_mou_6',
       ...
       'monthly_3g_9', 'sachet_3g_6', 'sachet_3g_7', 'sachet_3g_8',
       'sachet_3g_9', 'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g',
       'sep_vbc_3g'],
      dtype='object', length=166)
---------------------------------------------------------------------------------------------------- 

Non-numeric Columns 	:  Index(['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8',
       'date_of_last_rech_9'],
      dtype='object')
---------------------------------------------------------------------------------------------------- 



In [ ]:

##Treating all non numerical columns or Categorical Variables

There are 12 of them and all of these have more than 70% missing values. Missing values can be imputed with '-1' where -1 can be treated as a new category.

CATEGORICAL VARIABLES:

    night_pck_user_6
    night_pck_user_7
    night_pck_user_8
    night_pck_user_9
    fb_user_6
    fb_user_7
    fb_user_8
    fb_user_9



# Data Preparation

## 1. Derive new features
There are some importance variable which must have missing value. So, first lets create a new category.